## EU Driving regulation for truckers

The EU Regulation 561/2006 sets rules for driving time, breaks, and rest periods for drivers of goods vehicles over 3.5 tons. These regulations aim to improve road safety and protect drivers from excessive fatigue. This is a critical problem in the logistic industry that is actually handled by humans, manually defining the routes and breaks for the drivers. ...

Maximun Driving Time:
* Driving: 4.5 hours of continuous driving.
* Daily: 9 hours, extendable to 10 hours twice a week.
* Weekly: 56 hours.
* Bi-weekly: 90 hours in any two consecutive weeks.

Breaks:

* Driving: 45 minutes is mandatory (can be split into 15 minutes followed by 30 minutes) After 4.5 hours of continuous driving.
* Daily rest: 11 hours or reduced 9 hours (up to three times per week). After 9 hours of driving.
* Weekly rest: 45 hours or a reduced weekly rest period of at least 24 hours.



The tachograph records various driving and vehicle data, ensuring compliance with driving time, breaks, and rest period regulations. Information recorded by a tachograph includes:

* Driver's details: The driver's name, license number, and tachograph card number.
* Date and time: The device records date and time for all activities.
* Vehicle speed: The tachograph constantly tracks and records the vehicle's speed.
* Driving time: The device records the duration of driving sessions, including any breaks or rest periods.
* Distance traveled: The tachograph logs the distance covered during a driving session.
* Breaks and rest periods: The device records the duration and frequency of breaks and rest periods taken by the driver.
* Work mode: The tachograph differentiates between driving, working, availability, and rest modes.
* Events and faults: The device logs any events or faults, such as overspeeding, power supply interruptions, or sensor malfunctions.

# Example of a tachograph

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./data/driver1.csv")
status_translation = {
    'Pausa': 'Break',
    'Conduciendo': 'Driving',
    'Otro trabajo': 'Other work',
    'En espera': 'On standby'
}

# Replace the values in the Status column with the translated values
df['Status'] = df['Status'].map(status_translation)
df.head()

,Driver,DateStart,HourStart,HourEnd,Duration,Distance,Status,Profession,Vehicle,PlaceStart,PlaceEnd,LatitudeStart,LongitudeStart,LatitudeEnd,LongitudeEnd,MilometerStart,MilometerEnd
0,driver1,lun 2/01/17,17:59,18:13,0h 14m,0.028144,Break,1,9359JSNMI6186H01502,"28914, Leganés, ES","28914, Leganés, ES",40.327719,-3.784834,40.327719,-3.784502,"31,512","31,512"
1,driver1,lun 2/01/17,18:13,18:16,0h 03m,0.124866,Driving,1,9359JSNMI6186H01502,"28914, Leganés, ES","28914, Leganés, ES",40.327926,-3.784502,40.327926,-3.785975,"31,512","31,513"
2,driver1,lun 2/01/17,18:16,18:18,0h 02m,0.000509,Other work,1,9359JSNMI6186H01502,"28914, Leganés, ES","28914, Leganés, ES",40.329214,-3.785975,40.329214,-3.785981,"31,513","31,513"
3,driver1,lun 2/01/17,18:18,18:20,0h 02m,0.230232,Driving,1,9359JSNMI6186H01502,"28914, Leganés, ES","28914, Leganés, ES",40.328715,-3.785981,40.328715,-3.788697,"31,513","31,513"
4,driver1,lun 2/01/17,18:20,18:43,0h 23m,0.001611,Other work,1,9359JSNMI6186H01502,"28914, Leganés, ES","28914, Leganés, ES",40.328620,-3.788697,40.328620,-3.788678,"31,513","31,513"


In [28]:
df['Status'].unique()

array(['Break', 'Driving', 'Other work', 'On standby'], dtype=object)

# Solving it with UPF

In [3]:
%%capture
%pip install ..
%pip install ../unified-planning

In [4]:
from up_siadex import SIADEXEngine

import unified_planning as up
from unified_planning.shortcuts import *
from unified_planning.model.htn.hierarchical_problem import HierarchicalProblem, Task, Method
from unified_planning.io import PDDLReader
from unified_planning.io import PDDLWriter
from unified_planning.io.hpdl.hpdl_reader import HPDLReader
from unified_planning.io.hpdl.hpdl_writer import HPDLWriter
from unified_planning.engines.results import PlanGenerationResultStatus


In [5]:
env = up.environment.get_env()
env.factory.add_engine('siadex', __name__, "SIADEXEngine")

In [6]:
reader = HPDLReader()
problem = reader.parse_problem("../unified-planning/unified_planning/test/pddl/hpdl/jacob/domain.hpdl","../unified-planning/unified_planning/test/pddl/hpdl/jacob/problem.hpdl")

# A look to the problem 

In [29]:
action_status = {
"d_p": "Driving",
"o_p": "Other Work",
"b_p": "Break",
"i_p": "Idle",
"load_p": "Load Package",
"unload_p": "Unload Package",
"drive_p": "Drive to Destination",
"drive_p_fixed_duration": "Drive with Fixed Duration",
"refuel": "Refuel",
"refuel-on-the-way": "Refuel on the Way",
"d_suggested": "Suggested Driving",
"b_suggested": "Suggested Break"
}

In [30]:
with env.factory.OneshotPlanner(name='siadex') as p:
    result = p.solve(problem)
    print(result)
    if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
        print(f'{p.name()} found a valid plan!')
        print(f'The plan is: \n')
        for i,a in enumerate(result.plan.timed_actions):
            action_name = a[1].action.name
            if a[1].action.name in action_status:
                action_name = action_status[action_name]
            print(f"""Start: {a[0]} - End: {a[0] + a[2]} \n {action_name}({a[1].actual_parameters})""")
            print(f"_"*50)
    else:
        print('No plan found!')

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 1 of `/tmp/ipykernel_750/2485395871.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE



/home/gorgue/.local/lib/python3.10/site-packages/unified_planning/engines/mixins/oneshot_planner.py:73: UserWarning: <function SIADEXEngine.name at 0x7f0d88d691b0> cannot solve this kind of problem!
  warn(msg)


PlanGenerationResult(status=<PlanGenerationResultStatus.SOLVED_SATISFICING: 1>, plan=[(0.0, b_p(driver, 5, b_t0, split_1, first, ndd, 1, 1, yes), 5.0), (5.0, d_p(driver, 51, a, split_1, first, ndd, 1, 1, yes), 51.0), (56.0, b_p(driver, 137, b_t2, split_1, first, ndd, 1, 1, yes), 137.0), (193.0, d_p(driver, 2, a, split_2, first, ndd, 1, 1, yes), 2.0), (195.0, o_p(driver, 3, a, split_2, first, ndd, 1, 1, yes), 3.0), (198.0, d_p(driver, 2, a, split_2, first, ndd, 1, 1, yes), 2.0), (200.0, o_p(driver, 2, a, split_2, first, ndd, 1, 1, yes), 2.0), (202.0, b_p(driver, 71, b_t3, split_2, first, ndd, 1, 1, yes), 71.0), (273.0, d_p(driver, 1, a, split_1, second, ndd, 1, 1, yes), 1.0), (274.0, o_p(driver, 4, a, split_1, second, ndd, 1, 1, yes), 4.0), (278.0, d_p(driver, 49, a, split_1, second, ndd, 1, 1, yes), 49.0), (327.0, o_p(driver, 10, a, split_1, second, ndd, 1, 1, yes), 10.0), (337.0, d_p(driver, 60, a, split_1, second, ndd, 1, 1, yes), 60.0), (397.0, o_p(driver, 5, a, split_1, second, ndd